### Устанавливаем требуемые библиотеки

In [ ]:
#!pip install gradio scikit-learn pandas streamlit streamlit-jupyter

### Тренируем модель на датасете Iris

In [1]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Загрузка данных
iris = load_iris()
X = pd.DataFrame(iris.data, columns=iris.feature_names)
y = iris.target

# Разделение на тренировочную и тестовую выборку
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Обучение модели
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Проверка точности
accuracy = model.score(X_test, y_test)
print(f"Точность модели: {accuracy:.2f}")

Точность модели: 1.00


### Создание функции для предсказания
Функция будет принимать 4 параметра (длина/ширина чашелистика и лепестка) и возвращать название вида цветка.

In [3]:
def predict_species(sepal_length, sepal_width, petal_length, petal_width):
    # Входные данные -> DataFrame
    input_data = pd.DataFrame([[sepal_length, sepal_width, petal_length, petal_width]], 
                             columns=iris.feature_names)
    
    # Предсказание
    prediction = model.predict(input_data)[0]
    species = iris.target_names[prediction]
    
    # Вероятности для каждого класса
    probas = model.predict_proba(input_data)[0]
    confidence = {iris.target_names[i]: f"{probas[i]*100:.1f}%" for i in range(3)}
    
    return f"Вид: {species} (Уверенность: {confidence})"

### Запуск Gradio-интерфейса

In [5]:
import gradio as gr

# Описание входных полей
inputs = [
    gr.Slider(4.0, 8.0, step=0.1, label="Длина чашелистика (см)"),
    gr.Slider(2.0, 4.5, step=0.1, label="Ширина чашелистика (см)"),
    gr.Slider(1.0, 7.0, step=0.1, label="Длина лепестка (см)"),
    gr.Slider(0.1, 2.5, step=0.1, label="Ширина лепестка (см)")
]

# Заголовок и примеры
title = "Классификация цветков Iris"
examples = [
    [5.1, 3.5, 1.4, 0.2],  # setosa
    [6.7, 3.0, 5.2, 2.3],  # virginica
    [5.9, 3.0, 4.2, 1.5]   # versicolor
]

# Создание интерфейса
demo = gr.Interface(
    fn=predict_species,
    inputs=inputs,
    outputs="text",
    title=title,
    examples=examples,
    description="Введите параметры цветка для определения его вида."
)

# Запуск (в Jupyter используйте inline=True)
demo.launch(inline=True)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


### Запуск  Streamlit-интерфейса

In [7]:
%%writefile iris_streamlit_app.py
# 1. Установка необходимых библиотек (выполнить в терминале)
# pip install streamlit scikit-learn pandas

# 2. Создаем файл iris_app.py
import streamlit as st
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Загрузка данных
iris = load_iris()
X = pd.DataFrame(iris.data, columns=iris.feature_names)
y = iris.target

# Разделение данных
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Обучение модели
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Функция для предсказания
def predict_species(sepal_length, sepal_width, petal_length, petal_width):
    input_data = pd.DataFrame([[sepal_length, sepal_width, petal_length, petal_width]], 
                             columns=iris.feature_names)
    prediction = model.predict(input_data)[0]
    species = iris.target_names[prediction]
    probas = model.predict_proba(input_data)[0]
    return species, probas

# Интерфейс Streamlit
st.title('Классификация цветков Iris')
st.write('Это приложение предсказывает вид цветка ириса по его параметрам')

# Сайдбар с параметрами
st.sidebar.header('Параметры цветка')
sepal_length = st.sidebar.slider('Длина чашелистика (см)', 4.0, 8.0, 5.8)
sepal_width = st.sidebar.slider('Ширина чашелистика (см)', 2.0, 4.5, 3.0)
petal_length = st.sidebar.slider('Длина лепестка (см)', 1.0, 7.0, 4.0)
petal_width = st.sidebar.slider('Ширина лепестка (см)', 0.1, 2.5, 1.2)

# Предсказание
if st.sidebar.button('Предсказать'):
    species, probas = predict_species(sepal_length, sepal_width, petal_length, petal_width)
    
    st.subheader('Результат предсказания')
    st.success(f'Вид цветка: **{species}**')
    
    st.subheader('Вероятности классов')
    for i, prob in enumerate(probas):
        st.info(f'{iris.target_names[i]}: {prob*100:.1f}%')

# Информация о датасете
if st.checkbox('Показать информацию о датасете'):
    st.subheader('Датасет Iris')
    st.write(iris.DESCR)
    st.write('Первые 5 строк данных:')
    st.write(X.head())

Overwriting iris_streamlit_app.py


In [ ]:
!streamlit run iris_streamlit_app.py


  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://192.168.0.102:8501

